***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [4]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'mybucketbucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [6]:
# Count number of wiki pages
parquetFile.count()

6348910

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

23/01/05 16:35:32 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [13]:
from inverted_index_gcp import InvertedIndex

<h1>Doc Id - Title dictionary</h1>


In [ ]:
# doc_id_title_dict creation
doc_id_title_pairs = parquetFile.select("id", "title").rdd
doc_id_title_dict = doc_id_title_pairs.collectAsMap()

with open('doc_id_title_dict.pickle', 'wb') as f:
    pickle.dump(doc_id_title_dict, f)

index_src = "doc_id_title_dict.pickle"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

!gsutil ls -lh $index_dst

<H1> Helper function to create Indexes <H1/>

In [14]:
index_type = "_title"

In [15]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    cnt = Counter([tok for tok in tokens if tok not in all_stopwords])
    return [(key, (id, val)) for key,val in cnt.items()]
    
    
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key= lambda tup: tup[0])


def calculate_df(postings):
    return postings.map(lambda lm: (lm[0],len(lm[1])))


def partition_postings_and_write(postings):
    partition_rdd = postings.map(lambda w_pl: (token2bucket_id(w_pl[0]),w_pl)).groupByKey()
    return partition_rdd.map(lambda b_w_pl: InvertedIndex.write_a_posting_list(b_w_pl,bucket_name,index_type=index_type))

def word_count_binary(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    words_set = set(tok for tok in tokens if tok not in all_stopwords)
    return [(word, (id, 1)) for word in words_set]

def doc_len_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    return (id,len(tokens))

# Assignment2 Code: 

def tokenize(text):
  return [token.group() for token in RE_WORD.finditer(text.lower())]

def calculate_tf(docs):
  word_counts = Counter()
  for text in docs:
    tokens = tokenize(title)
    doc_counts = Counter(tokens)
    word_counts.update(doc_counts)
  return word_counts

<H1><c>Creating title Index<H1/>

In [ ]:
# word counts map

word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=f'postings_gcp{index_type}'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    
# Create inverted index instance
title_index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
title_index.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
title_index.df = w2df_title_dict
# write the global stats out
title_index.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/indexes/{index_src}'
!gsutil cp $index_src $index_dst

!gsutil ls -lh $index_dst

In [1]:
print(postings_title.take(1))

NameError: name 'postings_title' is not defined